# Couchbase Tutorial: SQL++

## In this tutorial, we'll cover how to use SQL++ for:
* Inserting Documents
* Listing All Documents In A Collection
* Updating Documents
* Querying Using WHERE
* Getting The Average Price Per Type By Aggregating Data Using GROUP BY
* Understand Query Execution By Using EXPLAIN
* Using Subqueries
* Inserting Documents With References To Existing Documents
* Using JOINs Between Collections
* Filtering By Values In Nested Objects
* Filtering Arrays
* Expanding Documents To Return One Row Per Array Entry Using UNNEST
* Full-Text Search: Setting Up An Index And Querying
* Get The Top 3 Most Expensive Products For Each Type Using The Window Functions DENSE_RANK And ROW_NUMBER
* Labeling Documents Using CASE Statements


In [1]:
%pip install couchbase

Note: you may need to restart the kernel to use updated packages.


## Set The Environment Variables

This is typically done in the polytope.yml file. Here we initiate the environment variables to simulate that they were already set by Polytope. 

In [2]:
%env COUCHBASE_USERNAME=admin
%env COUCHBASE_PASSWORD=password
%env COUCHBASE_BUCKET_NAME=main
%env COUCHBASE_URL=couchbase://couchbase

env: COUCHBASE_USERNAME=admin
env: COUCHBASE_PASSWORD=password
env: COUCHBASE_BUCKET_NAME=main
env: COUCHBASE_URL=couchbase://couchbase


## Module Import And Client Setup

In [4]:
import os
import uuid
import json
from datetime import timedelta
import couchbase.subdocument as SD
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import (ClusterOptions, ClusterTimeoutOptions, QueryOptions, MutateInOptions)
from couchbase.exceptions import ScopeAlreadyExistsException, CollectionAlreadyExistsException
from couchbase.management.collections import CreateCollectionSettings

COUCHBASE_USERNAME = os.environ['COUCHBASE_USERNAME']
COUCHBASE_PASSWORD = os.environ['COUCHBASE_PASSWORD']
COUCHBASE_BUCKET_NAME = os.environ['COUCHBASE_BUCKET_NAME']
COUCHBASE_URL = os.environ['COUCHBASE_URL']
# NOTE: For TLS/SSL connection use 'couchbases://' instead

auth = PasswordAuthenticator(
    COUCHBASE_USERNAME,
    COUCHBASE_PASSWORD
)
cluster = Cluster(COUCHBASE_URL, ClusterOptions(auth))
cluster.wait_until_ready(timedelta(seconds=5))
bucket = cluster.bucket(COUCHBASE_BUCKET_NAME)

## Prepare The Example Scope And Collections

In [5]:
def create_scope_and_collections(scope_name, collection_names):
    collection_manager = bucket.collections()

    # Create scope
    try:
        collection_manager.create_scope(scope_name)
        print(f"Scope '{scope_name}' created successfully.")
    except ScopeAlreadyExistsException:
        print(f"Scope '{scope_name}' already exists.")
    except Exception as e:
        print(f"An error occurred while creating scope: {e}")

    # Create collections
    for collection_name in collection_names:
        try:
            collection_manager.create_collection(
                scope_name,
                collection_name,
                CreateCollectionSettings()
            )
            print(f"Collection '{collection_name}' created successfully in scope '{scope_name}'.")
        except CollectionAlreadyExistsException:
            print(f"Collection '{collection_name}' already exists in scope '{scope_name}'.")
        except Exception as e:
            print(f"An error occurred while creating collection '{collection_name}': {e}")

scope_name = "cillers_play"
collection_name_products = "products"
collection_name_orders = "orders"

create_scope_and_collections(scope_name, [
    collection_name_products,
    collection_name_orders
])

Scope 'cillers_play' already exists.
Collection 'products' already exists in scope 'cillers_play'.
Collection 'orders' already exists in scope 'cillers_play'.


## Inserting Documents

In [6]:
insert_products_query = f"""
INSERT INTO `{bucket.name}`.`{scope_name}`.`{collection_name_products}` (KEY, VALUE)
VALUES 
    ("product_" || UUID(), {{
        "name": "Smartphone X", 
        "type": "Electronics", 
        "price": 699.99, 
        "details": {{
            "color": "Midnight Blue",
            "storage": "128GB",
            "screen_size": "6.1 inches"
        }},
        "tags": ["smartphone", "5G", "high-resolution camera", "water-resistant"],
        "description": "A powerful smartphone with 5G capability and a high-resolution camera."
    }}),
    ("product_" || UUID(), {{
        "name": "Laptop Pro", 
        "type": "Electronics", 
        "price": 1299.99, 
        "details": {{
            "color": "Silver",
            "processor": "Intel i7",
            "ram": "16GB"
        }},
        "tags": ["laptop", "high-performance", "lightweight", "long battery life"],
        "description": "A powerful and lightweight laptop with long battery life, perfect for professionals on the go."
    }}),
    ("product_" || UUID(), {{
        "name": "Smart TV 4K", 
        "type": "Electronics", 
        "price": 799.99, 
        "details": {{
            "color": "Black",
            "screen_size": "55 inches",
            "resolution": "4K"
        }},
        "tags": ["TV", "4K", "smart", "HDR", "voice control"],
        "description": "A smart 4K TV with HDR and voice control for an immersive viewing experience."
    }}),
    ("product_" || UUID(), {{
        "name": "Wireless Earbuds", 
        "type": "Electronics", 
        "price": 159.99, 
        "details": {{
            "color": "White",
            "battery_life": "24 hours",
            "connectivity": "Bluetooth 5.0"
        }},
        "tags": ["earbuds", "wireless", "noise-cancelling", "water-resistant"],
        "description": "Lightweight and water-resistant wireless earbuds with excellent noise-cancelling capabilities."
    }}),
    ("product_" || UUID(), {{
        "name": "Coffee Maker Deluxe", 
        "type": "Appliances", 
        "price": 129.99, 
        "details": {{
            "color": "Stainless Steel",
            "capacity": "12 cups",
            "features": "Programmable"
        }},
        "tags": ["coffee maker", "programmable", "thermal carafe"],
        "description": "A programmable coffee maker with a thermal carafe to keep your coffee hot for hours."
    }})
RETURNING *
"""
insert_products_result = cluster.query(insert_products_query)
print("Inserted products:")
for row in insert_products_result:
    print(json.dumps(row, indent=2))

Inserted products:
{
  "products": {
    "description": "A powerful and lightweight laptop with long battery life, perfect for professionals on the go.",
    "details": {
      "color": "Silver",
      "processor": "Intel i7",
      "ram": "16GB"
    },
    "name": "Laptop Pro",
    "price": 1299.99,
    "tags": [
      "laptop",
      "high-performance",
      "lightweight",
      "long battery life"
    ],
    "type": "Electronics"
  }
}
{
  "products": {
    "description": "A powerful smartphone with 5G capability and a high-resolution camera.",
    "details": {
      "color": "Midnight Blue",
      "screen_size": "6.1 inches",
      "storage": "128GB"
    },
    "name": "Smartphone X",
    "price": 699.99,
    "tags": [
      "smartphone",
      "5G",
      "high-resolution camera",
      "water-resistant"
    ],
    "type": "Electronics"
  }
}
{
  "products": {
    "description": "Lightweight and water-resistant wireless earbuds with excellent noise-cancelling capabilities.",
    

## Listing All Documents In A Collection

In [8]:
query_string = f"""
SELECT META(`{collection_name_products}`).id, *
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}` AS `{collection_name_products}`
"""
query_result = cluster.query(query_string)
for row in query_result:
    print(json.dumps(row, indent=2))

{
  "id": "product_a59c5bda-a001-473a-a676-96ff95b7b975",
  "products": {
    "description": "A powerful and lightweight laptop with long battery life, perfect for professionals on the go.",
    "details": {
      "color": "Silver",
      "processor": "Intel i7",
      "ram": "16GB"
    },
    "name": "Laptop Pro",
    "price": 1299.99,
    "tags": [
      "laptop",
      "high-performance",
      "lightweight",
      "long battery life"
    ],
    "type": "Electronics"
  }
}
{
  "id": "56ddbaea-768c-11ef-958f-0242ac11000f",
  "products": {
    "name": "Jupyter A"
  }
}
{
  "id": "product_02831be9-e38b-4cf8-b0b1-e5bdfcfaa52d",
  "products": {
    "description": "Lightweight and water-resistant wireless earbuds with excellent noise-cancelling capabilities.",
    "details": {
      "battery_life": "24 hours",
      "color": "White",
      "connectivity": "Bluetooth 5.0"
    },
    "name": "Wireless Earbuds",
    "price": 159.99,
    "tags": [
      "earbuds",
      "wireless",
      "noi

## Updating Documents

In [9]:
update_query = f"""
UPDATE `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
SET price = price * 1.1
WHERE type = "Appliances"
RETURNING *
"""
update_result = cluster.query(update_query)
for row in update_result:
    print(json.dumps(row, indent=2))

{
  "products": {
    "description": "A programmable coffee maker with a thermal carafe to keep your coffee hot for hours.",
    "details": {
      "capacity": "12 cups",
      "color": "Stainless Steel",
      "features": "Programmable"
    },
    "name": "Coffee Maker Deluxe",
    "price": 157.28790000000004,
    "tags": [
      "coffee maker",
      "programmable",
      "thermal carafe"
    ],
    "type": "Appliances"
  }
}
{
  "products": {
    "description": "A programmable coffee maker with a thermal carafe to keep your coffee hot for hours.",
    "details": {
      "capacity": "12 cups",
      "color": "Stainless Steel",
      "features": "Programmable"
    },
    "name": "Coffee Maker Deluxe",
    "price": 142.98900000000003,
    "tags": [
      "coffee maker",
      "programmable",
      "thermal carafe"
    ],
    "type": "Appliances"
  }
}


## Querying Using WHERE

In [10]:
condition_query = f"""
SELECT name, price
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
WHERE price > 700 AND type = "Electronics"
"""
condition_result = cluster.query(condition_query)
for row in condition_result:
    print(row)

{'name': 'Smart TV 4K', 'price': 799.99}
{'name': 'Smart TV 4K', 'price': 799.99}
{'name': 'Laptop Pro', 'price': 1299.99}
{'name': 'Laptop Pro', 'price': 1299.99}


## Getting The Average Price Per Type By Aggregating Data Using GROUP BY

In [11]:
aggregate_query = f"""
SELECT type, COUNT(*) as count, AVG(price) as avg_price
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
GROUP BY type
"""
aggregate_result = cluster.query(aggregate_query)
for row in aggregate_result:
    print(row)

{'avg_price': None, 'count': 1}
{'avg_price': 739.9899999999999, 'count': 8, 'type': 'Electronics'}
{'avg_price': 150.13845000000003, 'count': 2, 'type': 'Appliances'}


## Understand Query Execution By Using EXPLAIN

In [12]:
explain_query = f"""
EXPLAIN SELECT *
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
WHERE price > 100
"""
explain_result = cluster.query(explain_query)
print("Explain Plan:")
for row in explain_result:
    formatted_json = json.dumps(row, indent=2)
    print(formatted_json)

Explain Plan:
{
  "plan": {
    "#operator": "Sequence",
    "~children": [
      {
        "#operator": "PrimaryScan3",
        "bucket": "main",
        "index": "#sequentialscan",
        "index_projection": {
          "primary_key": true
        },
        "keyspace": "products",
        "namespace": "default",
        "scope": "cillers_play",
        "using": "sequentialscan"
      },
      {
        "#operator": "Fetch",
        "bucket": "main",
        "keyspace": "products",
        "namespace": "default",
        "scope": "cillers_play"
      },
      {
        "#operator": "Parallel",
        "~child": {
          "#operator": "Sequence",
          "~children": [
            {
              "#operator": "Filter",
              "condition": "(100 < (`products`.`price`))"
            },
            {
              "#operator": "InitialProject",
              "discard_original": true,
              "preserve_order": true,
              "result_terms": [
                {
     

## Using Subqueries

In [13]:
subquery = f"""
WITH avg_price AS (
    SELECT AVG(price) as avg
    FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
)
SELECT p.name, p.price
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}` p, avg_price
WHERE p.price > avg_price.avg
ORDER BY p.price DESC
"""
subquery_result = cluster.query(subquery)
print("Products above average price:")
for row in subquery_result:
    print(json.dumps(row, indent=2))

Products above average price:
{
  "name": "Laptop Pro",
  "price": 1299.99
}
{
  "name": "Laptop Pro",
  "price": 1299.99
}
{
  "name": "Smart TV 4K",
  "price": 799.99
}
{
  "name": "Smart TV 4K",
  "price": 799.99
}
{
  "name": "Smartphone X",
  "price": 699.99
}
{
  "name": "Smartphone X",
  "price": 699.99
}


## Inserting Documents With References To Existing Documents

In [14]:
# Fetch products
fetch_products_query = f"""
SELECT META().id as product_id, name
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
"""
products_result = cluster.query(fetch_products_query)
products = list(products_result)

# Insert orders
insert_orders_query = f"""
INSERT INTO `{bucket.name}`.`{scope_name}`.`{collection_name_orders}` (KEY, VALUE)
VALUES 
"""

for product in products:
    insert_orders_query += f"""
    ("order_" || UUID(), {{
        "order_id": "order_" || UUID(),
        "product_id": "{product['product_id']}",
        "quantity": 1
    }}),
    """

# Remove the trailing comma and add RETURNING clause
insert_orders_query = insert_orders_query.rstrip(',\n ') + "\nRETURNING *"

insert_orders_result = cluster.query(insert_orders_query)
print("\nInserted orders:")
for row in insert_orders_result:
    print(row)


Inserted orders:
{'orders': {'order_id': 'order_dc303a78-0521-4931-b574-c9186d037993', 'product_id': 'product_3fd4fc88-6ced-46e5-aa3d-9cb6163edafd', 'quantity': 1}}
{'orders': {'order_id': 'order_950a51c4-8d1e-4b87-8bf5-fcdf5705c8be', 'product_id': '56ddbaea-768c-11ef-958f-0242ac11000f', 'quantity': 1}}
{'orders': {'order_id': 'order_edea0674-dcf0-4027-ac9c-daf516bbf44e', 'product_id': 'product_a59c5bda-a001-473a-a676-96ff95b7b975', 'quantity': 1}}
{'orders': {'order_id': 'order_7d197916-9f56-42f9-9fc9-755a7b921ad7', 'product_id': 'product_02831be9-e38b-4cf8-b0b1-e5bdfcfaa52d', 'quantity': 1}}
{'orders': {'order_id': 'order_82ecbbbd-3036-4c9d-83a6-649927a12608', 'product_id': 'product_8dedf1d6-0f70-4093-b8ae-7c5587116090', 'quantity': 1}}
{'orders': {'order_id': 'order_bd41eba7-ed3a-4558-8a9e-857820366180', 'product_id': 'product_7a605790-acd9-49cf-8337-a46e76beaedc', 'quantity': 1}}
{'orders': {'order_id': 'order_cc2ba784-e264-4f67-835c-29dadd538cc5', 'product_id': 'product_80661d25-

## Using JOINs Between Collections

In [15]:
join_query = f"""
SELECT p.name, p.price, o.order_id, o.customer_name
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_orders}` AS o
JOIN `{bucket.name}`.`{scope_name}`.`{collection_name_products}` AS p
ON KEYS o.product_id
WHERE p.price > 40
"""

join_result = cluster.query(join_query)

for row in join_result:
    print(row)

{'name': 'Coffee Maker Deluxe', 'order_id': 'order_bd41eba7-ed3a-4558-8a9e-857820366180', 'price': 157.28790000000004}
{'name': 'Laptop Pro', 'order_id': 'order_dd394ed4-c1ee-4c0b-8a88-07603e2d674d', 'price': 1299.99}
{'name': 'Wireless Earbuds', 'order_id': 'order_2336756f-7e31-4327-a13e-610906302349', 'price': 159.99}
{'name': 'Smart TV 4K', 'order_id': 'order_462de7dd-b20e-4924-9b96-ab8f7542ae44', 'price': 799.99}
{'name': 'Smartphone X', 'order_id': 'order_cc2ba784-e264-4f67-835c-29dadd538cc5', 'price': 699.99}
{'name': 'Smartphone X', 'order_id': 'order_a3cc3e82-afb2-4cd7-940b-60b37cdb4f9a', 'price': 699.99}
{'name': 'Smart TV 4K', 'order_id': 'order_ff560143-7277-46ad-911b-9a0a75023009', 'price': 799.99}
{'name': 'Smart TV 4K', 'order_id': 'order_dbc4f3cb-6125-48b9-98b0-3d88b1f9bd2f', 'price': 799.99}
{'name': 'Smartphone X', 'order_id': 'order_82ecbbbd-3036-4c9d-83a6-649927a12608', 'price': 699.99}
{'name': 'Wireless Earbuds', 'order_id': 'order_dc303a78-0521-4931-b574-c9186d037

## Filtering By Values In Nested Objects

In [16]:
nested_query = f"""
SELECT name, details.color, details.screen_size
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
WHERE details.color = "Black"
"""

print("Matching products:")
nested_result = cluster.query(nested_query)
for row in nested_result:
    print(row)

Matching products:
{'color': 'Black', 'name': 'Smart TV 4K', 'screen_size': '55 inches'}
{'color': 'Black', 'name': 'Smart TV 4K', 'screen_size': '55 inches'}


## Filtering Arrays

In [17]:
array_query = f"""
SELECT name, price,
       ARRAY v FOR v IN tags WHEN v IN ["water-resistant", "noise-cancelling", "high-performance"] END AS highlighted_features,
       ARRAY_LENGTH(tags) AS feature_count
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
WHERE type = "Electronics"
ORDER BY price DESC
"""
print("Products with highlighted features:")
array_result = cluster.query(array_query)
for row in array_result:
    print(row)

Products with highlighted features:
{'feature_count': 4, 'highlighted_features': ['high-performance'], 'name': 'Laptop Pro', 'price': 1299.99}
{'feature_count': 4, 'highlighted_features': ['high-performance'], 'name': 'Laptop Pro', 'price': 1299.99}
{'feature_count': 5, 'highlighted_features': [], 'name': 'Smart TV 4K', 'price': 799.99}
{'feature_count': 5, 'highlighted_features': [], 'name': 'Smart TV 4K', 'price': 799.99}
{'feature_count': 4, 'highlighted_features': ['water-resistant'], 'name': 'Smartphone X', 'price': 699.99}
{'feature_count': 4, 'highlighted_features': ['water-resistant'], 'name': 'Smartphone X', 'price': 699.99}
{'feature_count': 4, 'highlighted_features': ['noise-cancelling', 'water-resistant'], 'name': 'Wireless Earbuds', 'price': 159.99}
{'feature_count': 4, 'highlighted_features': ['noise-cancelling', 'water-resistant'], 'name': 'Wireless Earbuds', 'price': 159.99}


## Expanding Documents To Return One Row Per Array Entry Using UNNEST

In [19]:
unnest_query = f"""
SELECT p.name, t AS tag
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}` p
UNNEST p.tags t
"""

unnest_result = cluster.query(unnest_query)

for row in unnest_result:
    print(row)

{'name': 'Laptop Pro', 'tag': 'laptop'}
{'name': 'Laptop Pro', 'tag': 'high-performance'}
{'name': 'Laptop Pro', 'tag': 'lightweight'}
{'name': 'Laptop Pro', 'tag': 'long battery life'}
{'name': 'Smart TV 4K', 'tag': 'TV'}
{'name': 'Smart TV 4K', 'tag': '4K'}
{'name': 'Smart TV 4K', 'tag': 'smart'}
{'name': 'Smart TV 4K', 'tag': 'HDR'}
{'name': 'Smart TV 4K', 'tag': 'voice control'}
{'name': 'Laptop Pro', 'tag': 'laptop'}
{'name': 'Laptop Pro', 'tag': 'high-performance'}
{'name': 'Laptop Pro', 'tag': 'lightweight'}
{'name': 'Laptop Pro', 'tag': 'long battery life'}
{'name': 'Smartphone X', 'tag': 'smartphone'}
{'name': 'Smartphone X', 'tag': '5G'}
{'name': 'Smartphone X', 'tag': 'high-resolution camera'}
{'name': 'Smartphone X', 'tag': 'water-resistant'}
{'name': 'Wireless Earbuds', 'tag': 'earbuds'}
{'name': 'Wireless Earbuds', 'tag': 'wireless'}
{'name': 'Wireless Earbuds', 'tag': 'noise-cancelling'}
{'name': 'Wireless Earbuds', 'tag': 'water-resistant'}
{'name': 'Coffee Maker Deluxe

## Full-Text Search

### Create An Full-Text Search Index

In [20]:
create_fts_index_query = f"""
CREATE FULLTEXT INDEX idx_product_description 
ON `{bucket.name}`.`{scope_name}`.`{collection_name_products}`(description)
WITH {{
    "default_analyzer": "standard",
    "default_mapping": {{
        "enabled": false,
        "dynamic": false
    }},
    "type_field": "type",
    "store_dynamic_fields": false,
    "index_dynamic_fields": false
}}
"""
try:
    cluster.query(create_fts_index_query)
    print("Full-text search index created successfully.")
except Exception as e:
    print(f"Error creating full-text search index: {e}")

Full-text search index created successfully.


### Execute Full-Text Search Query

In [22]:
fts_query = f"""
SELECT name, description
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
WHERE SEARCH(description, "powerful AND lightweight")
"""
fts_result = cluster.query(fts_query)
for row in fts_result:
    print(row)

{'description': 'A powerful smartphone with 5G capability and a high-resolution camera.', 'name': 'Smartphone X'}
{'description': 'Lightweight and water-resistant wireless earbuds with excellent noise-cancelling capabilities.', 'name': 'Wireless Earbuds'}
{'description': 'Lightweight and water-resistant wireless earbuds with excellent noise-cancelling capabilities.', 'name': 'Wireless Earbuds'}
{'description': 'A powerful smartphone with 5G capability and a high-resolution camera.', 'name': 'Smartphone X'}
{'description': 'A powerful and lightweight laptop with long battery life, perfect for professionals on the go.', 'name': 'Laptop Pro'}
{'description': 'A powerful and lightweight laptop with long battery life, perfect for professionals on the go.', 'name': 'Laptop Pro'}


## Get The Top 3 Most Expensive Products For Each Type Using The Window Functions DENSE_RANK And ROW_NUMBER

Note, we want more than three products if there are multiple products tied for third place. 

In [23]:
window_query = f"""
WITH ranked_products AS (
    SELECT name, type, price,
           DENSE_RANK() OVER (PARTITION BY type ORDER BY price DESC) AS price_rank,
           ROW_NUMBER() OVER (PARTITION BY type ORDER BY price DESC) AS row_num
    FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
),
third_rank AS (
    SELECT type, MAX(price_rank) AS max_rank
    FROM ranked_products
    WHERE row_num = 3
    GROUP BY type
)
SELECT rp.name, rp.type, rp.price, rp.price_rank
FROM ranked_products rp
LEFT JOIN third_rank fr ON rp.type = fr.type
WHERE rp.price_rank <= COALESCE(fr.max_rank, rp.price_rank)
ORDER BY rp.type, rp.price_rank
"""
window_result = cluster.query(window_query)
print("The top 3 most expensive products for each type:")
for row in window_result:
    print(row)

The top 3 most expensive products for each type:
{'name': 'Jupyter A', 'price_rank': 1}
{'name': 'Coffee Maker Deluxe', 'price': 157.28790000000004, 'price_rank': 1, 'type': 'Appliances'}
{'name': 'Coffee Maker Deluxe', 'price': 142.98900000000003, 'price_rank': 2, 'type': 'Appliances'}
{'name': 'Laptop Pro', 'price': 1299.99, 'price_rank': 1, 'type': 'Electronics'}
{'name': 'Laptop Pro', 'price': 1299.99, 'price_rank': 1, 'type': 'Electronics'}
{'name': 'Smart TV 4K', 'price': 799.99, 'price_rank': 2, 'type': 'Electronics'}
{'name': 'Smart TV 4K', 'price': 799.99, 'price_rank': 2, 'type': 'Electronics'}


## Labeling Documents Using CASE Statements

In [25]:
case_query = f"""
SELECT name, price,
       CASE 
           WHEN price < 500 THEN "Budget"
           WHEN price >= 500 AND price < 1000 THEN "Mid-range"
           ELSE "Premium"
       END AS price_category
FROM `{bucket.name}`.`{scope_name}`.`{collection_name_products}`
ORDER BY CASE price_category
           WHEN "Budget" THEN 1
           WHEN "Mid-range" THEN 2
           WHEN "Premium" THEN 3
         END,
         price DESC
"""

case_result = cluster.query(case_query)

for row in case_result:
    print(row)

{'name': 'Wireless Earbuds', 'price': 159.99, 'price_category': 'Budget'}
{'name': 'Wireless Earbuds', 'price': 159.99, 'price_category': 'Budget'}
{'name': 'Coffee Maker Deluxe', 'price': 157.28790000000004, 'price_category': 'Budget'}
{'name': 'Coffee Maker Deluxe', 'price': 142.98900000000003, 'price_category': 'Budget'}
{'name': 'Smart TV 4K', 'price': 799.99, 'price_category': 'Mid-range'}
{'name': 'Smart TV 4K', 'price': 799.99, 'price_category': 'Mid-range'}
{'name': 'Smartphone X', 'price': 699.99, 'price_category': 'Mid-range'}
{'name': 'Smartphone X', 'price': 699.99, 'price_category': 'Mid-range'}
{'name': 'Laptop Pro', 'price': 1299.99, 'price_category': 'Premium'}
{'name': 'Laptop Pro', 'price': 1299.99, 'price_category': 'Premium'}
{'name': 'Jupyter A', 'price_category': 'Premium'}
